# 채권 분석 보고서 크롤링

In [ ]:
# 채권 분석자료 Crawling

import requests
from bs4 import BeautifulSoup
from tika import parser
import pandas as pd
from IPython.display import clear_output
    
start_page = 1
current_page = start_page
end_page = 126

result_df = pd.DataFrame(columns=['date', 'text'])

while True:
    URL = 'https://finance.naver.com/research/debenture_list.nhn?&page={}'.format(current_page)
    r = requests.get(URL)
    sp = BeautifulSoup(r.text, 'html.parser')
    results = sp.select('table.type_1 td.file')

    for i, result in enumerate(results):
        pdf_date = sp.select('table.type_1 td.date')[i*2].text
        pdf_temp = requests.get(result.select_one('a').attrs['href'])

        with open('bond.pdf', 'wb') as f:
            f.write(pdf_temp.content)
        
        parsed = parser.from_file('bond.pdf')
        temp_text = str(parsed['content']).strip()
        temp_df = pd.DataFrame([{'date':pdf_date, 'text':temp_text}])
        result_df = pd.concat([result_df, temp_df]) 
        
    clear_output(wait=True)
    print('크롤링 {}% 진행 중...'.format(int((current_page/end_page)*100)))
        
    if current_page == end_page:
        clear_output(wait=True)
        print('데이터 크롤링 완료 ! bond_datas.csv에서 확인 가능')
        result_df.to_csv('bond_datas.csv')
        break
    current_page += 1

In [ ]:
# result_df['text'][0]

# 데이터 불러오기 (&전처리)

In [20]:
import pandas as pd
bond_df = pd.read_csv('bond_datas.csv')
bond_df = bond_df[['date', 'text']]

# 전처리한 것
------------------------
시작 Microsoft
Microsoft[^\\]*


Compliance Notice
(당사는)? ?(동|본) ?(조사|자료).*(없습니다|않습니다|됩니다)\.

마지막 '주: 발표.*'  

그림, 표
\<[^\<\>]*\>

그림 [0-9]{1,2}\.
표 [0-9]{1,2}\.

------------------------
Header .*키움증권 리서치센터 투자전략팀
.*팀

ㅎㅎㅎ으로 시작


.*월간채권투자  [0-9]{1,2}월 - 페이지 나누는 기준
Footer 본사·서울시.*

Footer
성명 직급.*
(담당업종 전화 이메일)

필요없는 문장 (자료출처)
자료: [가-힣a-zA-Z, ]* 


이메일
[a-z0-9A-Z]*\@[a-z0-9A-Z]*\.[a-zA-Z]{2,3}\.?[a-zA-Z]{0,2}

---------------------
확인 필요
[가-힣 ]{3,10}[^가-힣]*[0-9]{3,4}\-[0-9]{3,4} [a-zA-Z]

.*Comment

In [ ]:
bond_df.iloc[4].text

시작 Microsoft [0-9 \-\_ \\a-zA-Z\.]*
마지막 'Compliance Notice .*'
마지막 주: 발표.*

In [ ]:
bond_df.iloc[7].text

Header .*키움증권 리서치센터 투자전략팀


In [ ]:
bond_df.iloc[6].text

In [ ]:
bond_df.iloc[3745].text

In [ ]:
bond_df.iloc[3751].text
ㅎㅎㅎ으로 시작

.*월간채권투자  [0-9]{1,2}월 - 페이지 나누는 기준
# Footer 본사·서울시.*

In [21]:
bond_df

,date,text
0,20.07.15,Microsoft Word - 2020071416393691.docx\n\n\n \...
1,20.07.15,2013년 0월 0일\n\n\n \n\n \n\n \n\n \n\n \n\n \n\...
2,20.07.14,(Microsoft Word - 20200714_Market Inside_\277\...
3,20.07.13,Compliance Notice \n\n \n\n \n\n당사는 동 자료를 기관투...
4,20.07.10,Microsoft Word - 2020070919412559.docx\n\n\n \...
...,...,...
3747,08.04.21,Microsoft Word - 920_0421_Fixed Income Weekly....
3748,08.04.14,Microsoft Word - Fixedincome0414.doc\n\n\n \n\...
3749,08.04.07,Microsoft Word - 0407.doc\n\n\nFixed Income We...
3750,08.04.07,Microsoft Word - 0407.doc\n\n\nFixed Income We...


In [22]:
from datetime import date
import datetime
bond_df['date'] = list(map(lambda i : datetime.datetime.strptime(i, "%y.%m.%d").date(), bond_df['date']))

bond_df

,date,text
0,2020-07-15,Microsoft Word - 2020071416393691.docx\n\n\n \...
1,2020-07-15,2013년 0월 0일\n\n\n \n\n \n\n \n\n \n\n \n\n \n\...
2,2020-07-14,(Microsoft Word - 20200714_Market Inside_\277\...
3,2020-07-13,Compliance Notice \n\n \n\n \n\n당사는 동 자료를 기관투...
4,2020-07-10,Microsoft Word - 2020070919412559.docx\n\n\n \...
...,...,...
3747,2008-04-21,Microsoft Word - 920_0421_Fixed Income Weekly....
3748,2008-04-14,Microsoft Word - Fixedincome0414.doc\n\n\n \n\...
3749,2008-04-07,Microsoft Word - 0407.doc\n\n\nFixed Income We...
3750,2008-04-07,Microsoft Word - 0407.doc\n\n\nFixed Income We...


In [24]:
import re

def pp(x):
    x = x.replace('\n','[!n]')
    x = re.sub('Microsoft[^\[\!n\]]*', '', x) # Header 1
    x = re.sub('.*키움증권 리서치센터 투자전략팀', '', x) # Header 2 (추가확인필요)
    x = re.sub('본사·서울시.*', '', x) # Footer 1
    x = re.sub('성명 직급.*', '', x) # Footer 2
    x = re.sub('Compliance Notice', '', x) # Notice
    x = re.sub('(당사는)? ?(동|본) ?(조사|자료).*(없습니다|않습니다|됩니다|바랍니다)\.?', '', x) # Notice
    x = re.sub('\<[^\<\>]*\>', '', x) # <>사이의 내용
    x = re.sub('그림 [0-9]{1,2}\.', '', x) # 그림
    x = re.sub('표 [0-9]{1,2}\.', '', x) # 표
    x = re.sub('월간채권투자  [0-9]{1,2}월.', '', x) # page 나누는 기준
    x = re.sub('자료: [가-힣a-zA-Z, ]* ?', '', x) # 자료 출처
    x = re.sub('[a-zA-Z0-9\.]*\@[a-zA-Z0-9\.]*', '', x) # 이메일
    x = x.replace('[!n]', ' ')
    return x

bond_df['doc'] = list(map(pp, bond_df['text']))

KeyboardInterrupt: 

In [16]:
bond_df[bond_df['date'] < datetime.date(2018,1,1)].sort_values(by='date').to_json('bond_0517.json')
bond_df[bond_df['date'] > datetime.date(2017,12,31)].sort_values(by='date').to_json('bond_1820.json')

NameError: name 'datetime' is not defined

In [30]:
import pandas as pd
bond_df = pd.read_csv('bond_0517dirty.csv')[['date','doc']]

In [31]:
bond_df.to_json('bond_0517dirty.json')

In [25]:
import re

def comma(x):
    x = re.sub('([0-9a-zA-Z]{1,100}\.[0-9a-zA-Z]{0,100}%?)', '', x)
    x = re.sub(' [가-힣]{1}\.', '', x)
    return x
    
bond_df['doc'] = list(map(comma, bond_df['doc']))

In [26]:
bond_df

,date,doc
0,2008-04-01,ㅎㅎㅎ 2008_04 월간채권투자 채 권 시 장 전 망 금 융 시 장 ...
1,2008-04-07,Fixed Income Weekly #918 Fixed Inco...
2,2008-04-07,Fixed Income Weekly #918 Fixed Inco...
3,2008-04-14,4월 금통위는 정부-한은간 policy mix 합의의 반...
4,2008-04-21,ncome 경기 전망은 비교적 분명… 인플레 명분이 시점...
...,...,...
3092,2017-12-18,( Credit Report 18 ...
3093,2017-12-20,Fixed Income 20 11월 금통위 의사록 ...
3094,2017-12-20,2013년 0월 0일 2금...
3095,2017-12-26,PowerPoint 프레젠테이션 Fixed Income Analyst 김지만 ...


In [13]:
sents = []
for data in bond_df.iloc:
    temp = data['doc'].split('.')
    for s in temp:
        s = s.strip()
        for i in range(3):
            s = s.replace('  ', ' ')
        if len(s) < 2:
            continue
        
        sents.append(s)

In [14]:
len(sents)

122017

In [ ]:
sents